# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [3]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [4]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [5]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [6]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
n_features = 296
baseline_model = Sequential()
baseline_model.add(layers.Dense(n_features, activation='relu'))
# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu'))
baseline_model.add(layers.Dense(50,activation='relu'))
# Hidden layer with 50 units


# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(np.array(X_train), 
                   np.array(y_train), 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 408us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 146us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 146us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 175us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 245us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 240us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150

1051/1051 [==============================] - ETA: 0s - loss: nan - mean_squared_error: n - 0s 166us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 53/150
1051/1051 [==============================] - 0s 161us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 116us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 130us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 149us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 147us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1

1051/1051 [==============================] - 0s 234us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 104/150
1051/1051 [==============================] - 0s 244us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 105/150
1051/1051 [==============================] - 0s 174us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 175us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 268us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 238us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [12]:
# Numeric column names
numeric_columns = list(X_train_numeric.columns)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [13]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [17]:
# Train the model 
normalized_input_model.fit(np.array(X_train), 
                   np.array(y_train), 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

ValueError: Error when checking input: expected dense_8_input to have shape (296,) but got array with shape (36,)

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [15]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [18]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(np.array(X_train), 
                   np.array(y_train_scaled), 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 390us/sample - loss: 0.4834 - mean_squared_error: 0.4834 - val_loss: 0.2138 - val_mean_squared_error: 0.2138
Epoch 2/150
1051/1051 [==============================] - 0s 128us/sample - loss: 0.2581 - mean_squared_error: 0.2581 - val_loss: 0.1881 - val_mean_squared_error: 0.1881
Epoch 3/150
1051/1051 [==============================] - 0s 179us/sample - loss: 0.2057 - mean_squared_error: 0.2057 - val_loss: 0.1678 - val_mean_squared_error: 0.1678
Epoch 4/150
1051/1051 [==============================] - ETA: 0s - loss: 0.1653 - mean_squared_error: 0.16 - 0s 219us/sample - loss: 0.1750 - mean_squared_error: 0.1750 - val_loss: 0.2044 - val_mean_squared_error: 0.2044
Epoch 5/150
1051/1051 [==============================] - 0s 224us/sample - loss: 0.1563 - mean_squared_error: 0.1563 - val_loss: 0.1558 - val_mean_squared_error: 0.1558
Epoch 6/150
1051/1051 [==============================] -

Epoch 49/150
1051/1051 [==============================] - 0s 138us/sample - loss: 0.0315 - mean_squared_error: 0.0315 - val_loss: 0.1578 - val_mean_squared_error: 0.1578
Epoch 50/150
1051/1051 [==============================] - 0s 133us/sample - loss: 0.0308 - mean_squared_error: 0.0308 - val_loss: 0.1612 - val_mean_squared_error: 0.1612
Epoch 51/150
1051/1051 [==============================] - 0s 232us/sample - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.1607 - val_mean_squared_error: 0.1607
Epoch 52/150
1051/1051 [==============================] - 0s 220us/sample - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1597 - val_mean_squared_error: 0.1597
Epoch 53/150
1051/1051 [==============================] - 0s 218us/sample - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1596 - val_mean_squared_error: 0.1596
Epoch 54/150
1051/1051 [==============================] - 0s 109us/sample - loss: 0.0285 - mean_squared_error: 0.0285 - val_loss: 0.1587 - val_mean_sq

Epoch 97/150
1051/1051 [==============================] - 0s 226us/sample - loss: 0.0165 - mean_squared_error: 0.0165 - val_loss: 0.1664 - val_mean_squared_error: 0.1664
Epoch 98/150
1051/1051 [==============================] - 0s 183us/sample - loss: 0.0161 - mean_squared_error: 0.0161 - val_loss: 0.1673 - val_mean_squared_error: 0.1673
Epoch 99/150
1051/1051 [==============================] - 0s 114us/sample - loss: 0.0161 - mean_squared_error: 0.0161 - val_loss: 0.1660 - val_mean_squared_error: 0.1660
Epoch 100/150
1051/1051 [==============================] - 0s 237us/sample - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.1659 - val_mean_squared_error: 0.1659
Epoch 101/150
1051/1051 [==============================] - 0s 148us/sample - loss: 0.0153 - mean_squared_error: 0.0153 - val_loss: 0.1658 - val_mean_squared_error: 0.1658
Epoch 102/150
1051/1051 [==============================] - 0s 101us/sample - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.1680 - val_mean

Epoch 145/150
1051/1051 [==============================] - 0s 102us/sample - loss: 0.0096 - mean_squared_error: 0.0096 - val_loss: 0.1686 - val_mean_squared_error: 0.1686
Epoch 146/150
1051/1051 [==============================] - 0s 157us/sample - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 0.1690 - val_mean_squared_error: 0.1690
Epoch 147/150
1051/1051 [==============================] - 0s 172us/sample - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.1679 - val_mean_squared_error: 0.1679
Epoch 148/150
1051/1051 [==============================] - 0s 211us/sample - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.1697 - val_mean_squared_error: 0.1697
Epoch 149/150
1051/1051 [==============================] - 0s 157us/sample - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.1690 - val_mean_squared_error: 0.1690
Epoch 150/150
1051/1051 [==============================] - 0s 178us/sample - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.1689 - val_m

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 30us/sample - loss: 0.0108 - mean_squared_error: 0.0108


[0.010819933837993899, 0.010819932]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [20]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 38us/sample - loss: 0.1689 - mean_squared_error: 0.1689


[0.16891657694211024, 0.16891658]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [30]:
np.sqrt(np.mean([x**2 for x in (y_val - y_val_pred)['SalePrice']]))

32297.405465666587

In [21]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
rms = np.sqrt(np.mean((y_val - y_val_pred)**2))
rms

SalePrice    32297.405466
dtype: float64

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [31]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape = (n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 394us/sample - loss: 0.6277 - mean_squared_error: 0.6277 - val_loss: 0.2125 - val_mean_squared_error: 0.2125
Epoch 2/150
1051/1051 [==============================] - 0s 105us/sample - loss: 0.2813 - mean_squared_error: 0.2813 - val_loss: 0.1652 - val_mean_squared_error: 0.1652
Epoch 3/150
1051/1051 [==============================] - 0s 136us/sample - loss: 0.2266 - mean_squared_error: 0.2266 - val_loss: 0.1471 - val_mean_squared_error: 0.1471
Epoch 4/150
1051/1051 [==============================] - 0s 132us/sample - loss: 0.1943 - mean_squared_error: 0.1943 - val_loss: 0.2050 - val_mean_squared_error: 0.2050
Epoch 5/150
1051/1051 [==============================] - 0s 162us/sample - loss: 0.1800 - mean_squared_error: 0.1800 - val_loss: 0.1358 - val_mean_squared_error: 0.1358
Epoch 6/150
1051/1051 [==============================] - 0s 228us/sample - loss: 0.1605 - mean_squared_error

Epoch 49/150
1051/1051 [==============================] - 0s 146us/sample - loss: 0.0416 - mean_squared_error: 0.0416 - val_loss: 0.1089 - val_mean_squared_error: 0.1089
Epoch 50/150
1051/1051 [==============================] - 0s 264us/sample - loss: 0.0404 - mean_squared_error: 0.0404 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 51/150
1051/1051 [==============================] - 0s 137us/sample - loss: 0.0391 - mean_squared_error: 0.0391 - val_loss: 0.1128 - val_mean_squared_error: 0.1128
Epoch 52/150
1051/1051 [==============================] - 0s 126us/sample - loss: 0.0396 - mean_squared_error: 0.0396 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 53/150
1051/1051 [==============================] - 0s 136us/sample - loss: 0.0380 - mean_squared_error: 0.0380 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 54/150
1051/1051 [==============================] - 0s 180us/sample - loss: 0.0372 - mean_squared_error: 0.0372 - val_loss: 0.1095 - val_mean_sq

1051/1051 [==============================] - 0s 126us/sample - loss: 0.0203 - mean_squared_error: 0.0203 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 97/150
1051/1051 [==============================] - 0s 143us/sample - loss: 0.0208 - mean_squared_error: 0.0208 - val_loss: 0.1176 - val_mean_squared_error: 0.1176
Epoch 98/150
1051/1051 [==============================] - 0s 120us/sample - loss: 0.0206 - mean_squared_error: 0.0206 - val_loss: 0.1149 - val_mean_squared_error: 0.1149
Epoch 99/150
1051/1051 [==============================] - 0s 161us/sample - loss: 0.0207 - mean_squared_error: 0.0207 - val_loss: 0.1133 - val_mean_squared_error: 0.1133
Epoch 100/150
1051/1051 [==============================] - 0s 93us/sample - loss: 0.0200 - mean_squared_error: 0.0200 - val_loss: 0.1144 - val_mean_squared_error: 0.1144
Epoch 101/150
1051/1051 [==============================] - 0s 150us/sample - loss: 0.0198 - mean_squared_error: 0.0198 - val_loss: 0.1148 - val_mean_squared_error:

1051/1051 [==============================] - 0s 110us/sample - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 145/150
1051/1051 [==============================] - 0s 142us/sample - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 146/150
1051/1051 [==============================] - 0s 130us/sample - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 147/150
1051/1051 [==============================] - 0s 128us/sample - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.1188 - val_mean_squared_error: 0.1188
Epoch 148/150
1051/1051 [==============================] - 0s 134us/sample - loss: 0.0131 - mean_squared_error: 0.0131 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 149/150
1051/1051 [==============================] - 0s 121us/sample - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1186 - val_mean_squared_er

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/sample - loss: 0.0116 - mean_squared_error: 0.0116


[0.01162230696876513, 0.011622308]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on validate data
he_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 55us/sample - loss: 0.1784 - mean_squared_error: 0.1784


[0.1784051828188439, 0.17840518]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [34]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape = (n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 423us/sample - loss: 0.4951 - mean_squared_error: 0.4951 - val_loss: 0.1841 - val_mean_squared_error: 0.1841
Epoch 2/150
1051/1051 [==============================] - 0s 84us/sample - loss: 0.2713 - mean_squared_error: 0.2713 - val_loss: 0.1588 - val_mean_squared_error: 0.1588
Epoch 3/150
1051/1051 [==============================] - 0s 149us/sample - loss: 0.2191 - mean_squared_error: 0.2191 - val_loss: 0.1262 - val_mean_squared_error: 0.1262
Epoch 4/150
1051/1051 [==============================] - 0s 191us/sample - loss: 0.1804 - mean_squared_error: 0.1804 - val_loss: 0.1591 - val_mean_squared_error: 0.1591
Epoch 5/150
1051/1051 [==============================] - 0s 218us/sample - loss: 0.1653 - mean_squared_error: 0.1653 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 6/150
1051/1051 [==============================] - 0s 228us/sample - loss: 0.1441 - mean_squared_error:

Epoch 49/150
1051/1051 [==============================] - 0s 217us/sample - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 50/150
1051/1051 [==============================] - 0s 206us/sample - loss: 0.0313 - mean_squared_error: 0.0313 - val_loss: 0.1047 - val_mean_squared_error: 0.1047
Epoch 51/150
1051/1051 [==============================] - 0s 199us/sample - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.1039 - val_mean_squared_error: 0.1039
Epoch 52/150
1051/1051 [==============================] - 0s 207us/sample - loss: 0.0307 - mean_squared_error: 0.0307 - val_loss: 0.1044 - val_mean_squared_error: 0.1044
Epoch 53/150
1051/1051 [==============================] - 0s 192us/sample - loss: 0.0290 - mean_squared_error: 0.0290 - val_loss: 0.1054 - val_mean_squared_error: 0.1054
Epoch 54/150
1051/1051 [==============================] - 0s 87us/sample - loss: 0.0284 - mean_squared_error: 0.0284 - val_loss: 0.1034 - val_mean_squ

1051/1051 [==============================] - 0s 197us/sample - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.1074 - val_mean_squared_error: 0.1074
Epoch 98/150
1051/1051 [==============================] - 0s 212us/sample - loss: 0.0149 - mean_squared_error: 0.0149 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 99/150
1051/1051 [==============================] - 0s 234us/sample - loss: 0.0147 - mean_squared_error: 0.0147 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 100/150
1051/1051 [==============================] - 0s 204us/sample - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1076 - val_mean_squared_error: 0.1076
Epoch 101/150
1051/1051 [==============================] - 0s 226us/sample - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1048 - val_mean_squared_error: 0.1048
Epoch 102/150
1051/1051 [==============================] - 0s 224us/sample - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1068 - val_mean_squared_erro

1051/1051 [==============================] - 0s 212us/sample - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.1062 - val_mean_squared_error: 0.1062
Epoch 146/150
1051/1051 [==============================] - 0s 196us/sample - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.1066 - val_mean_squared_error: 0.1066
Epoch 147/150
1051/1051 [==============================] - 0s 209us/sample - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 148/150
1051/1051 [==============================] - 0s 196us/sample - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.1065 - val_mean_squared_error: 0.1065
Epoch 149/150
1051/1051 [==============================] - 0s 206us/sample - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1079 - val_mean_squared_error: 0.1079
Epoch 150/150
1051/1051 [==============================] - 0s 215us/sample - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1066 - val_mean_squared_er

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 121us/sample - loss: 0.0079 - mean_squared_error: 0.0079


[0.007938737513144792, 0.007938736]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [36]:
# Evaluate the model on validate data
lecun_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 62us/sample - loss: 0.1990 - mean_squared_error: 0.1990


[0.19900590671251897, 0.19900592]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [38]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(
    optimizer = 'rmsprop',
    loss='mse', 
    metrics=['mse'] 
)

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 571us/sample - loss: 0.5062 - mean_squared_error: 0.5062 - val_loss: 0.1281 - val_mean_squared_error: 0.1281
Epoch 2/150
1051/1051 [==============================] - ETA: 0s - loss: 0.1727 - mean_squared_error: 0.17 - 0s 170us/sample - loss: 0.1799 - mean_squared_error: 0.1799 - val_loss: 0.1429 - val_mean_squared_error: 0.1429
Epoch 3/150
1051/1051 [==============================] - 0s 178us/sample - loss: 0.1296 - mean_squared_error: 0.1296 - val_loss: 0.1062 - val_mean_squared_error: 0.1062
Epoch 4/150
1051/1051 [==============================] - 0s 172us/sample - loss: 0.1055 - mean_squared_error: 0.1055 - val_loss: 0.1348 - val_mean_squared_error: 0.1348
Epoch 5/150
1051/1051 [==============================] - 0s 160us/sample - loss: 0.0833 - mean_squared_error: 0.0833 - val_loss: 0.1139 - val_mean_squared_error: 0.1139
Epoch 6/150
1051/1051 [==============================] -

Epoch 49/150
1051/1051 [==============================] - 0s 230us/sample - loss: 0.0156 - mean_squared_error: 0.0156 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 50/150
1051/1051 [==============================] - 0s 240us/sample - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.1143 - val_mean_squared_error: 0.1143
Epoch 51/150
1051/1051 [==============================] - 0s 239us/sample - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.1122 - val_mean_squared_error: 0.1122
Epoch 52/150
1051/1051 [==============================] - 0s 243us/sample - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 53/150
1051/1051 [==============================] - 0s 246us/sample - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1041 - val_mean_squared_error: 0.1041
Epoch 54/150
1051/1051 [==============================] - 0s 235us/sample - loss: 0.0109 - mean_squared_error: 0.0109 - val_loss: 0.1000 - val_mean_sq

1051/1051 [==============================] - 0s 231us/sample - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.1035 - val_mean_squared_error: 0.1035
Epoch 97/150
1051/1051 [==============================] - 0s 267us/sample - loss: 0.0107 - mean_squared_error: 0.0107 - val_loss: 0.1037 - val_mean_squared_error: 0.1037
Epoch 98/150
1051/1051 [==============================] - 0s 221us/sample - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0957 - val_mean_squared_error: 0.0957
Epoch 99/150
1051/1051 [==============================] - 0s 132us/sample - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.1057 - val_mean_squared_error: 0.1057
Epoch 100/150
1051/1051 [==============================] - 0s 244us/sample - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.0966 - val_mean_squared_error: 0.0966
Epoch 101/150
1051/1051 [==============================] - 0s 231us/sample - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.1044 - val_mean_squared_error

1051/1051 [==============================] - 0s 214us/sample - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0970 - val_mean_squared_error: 0.0970
Epoch 145/150
1051/1051 [==============================] - 0s 247us/sample - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.1086 - val_mean_squared_error: 0.1086
Epoch 146/150
1051/1051 [==============================] - 0s 235us/sample - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0946 - val_mean_squared_error: 0.0946
Epoch 147/150
1051/1051 [==============================] - 0s 118us/sample - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0885 - val_mean_squared_error: 0.0885
Epoch 148/150
1051/1051 [==============================] - 0s 225us/sample - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.1037 - val_mean_squared_error: 0.1037
Epoch 149/150
1051/1051 [==============================] - 0s 241us/sample - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0961 - val_mean_squared_er

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [39]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 93us/sample - loss: 0.0056 - mean_squared_error: 0.0056


[0.00555376184273061, 0.005553762]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [40]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 123us/sample - loss: 0.1765 - mean_squared_error: 0.1765


[0.17654668876569565, 0.1765467]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [41]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(
    optimizer = 'adam',
    loss='mse', 
    metrics=['mse'] 
)

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 544us/sample - loss: 0.3639 - mean_squared_error: 0.3639 - val_loss: 0.1520 - val_mean_squared_error: 0.1520
Epoch 2/150
1051/1051 [==============================] - 0s 143us/sample - loss: 0.1595 - mean_squared_error: 0.1595 - val_loss: 0.1208 - val_mean_squared_error: 0.1208
Epoch 3/150
1051/1051 [==============================] - 0s 145us/sample - loss: 0.1041 - mean_squared_error: 0.1041 - val_loss: 0.1052 - val_mean_squared_error: 0.1052
Epoch 4/150
1051/1051 [==============================] - 0s 145us/sample - loss: 0.0817 - mean_squared_error: 0.0817 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 5/150
1051/1051 [==============================] - 0s 148us/sample - loss: 0.0589 - mean_squared_error: 0.0589 - val_loss: 0.1115 - val_mean_squared_error: 0.1115
Epoch 6/150
1051/1051 [==============================] - 0s 237us/sample - loss: 0.0521 - mean_squared_error

Epoch 49/150
1051/1051 [==============================] - 0s 162us/sample - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.1200 - val_mean_squared_error: 0.1200
Epoch 50/150
1051/1051 [==============================] - 0s 161us/sample - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.1062 - val_mean_squared_error: 0.1062
Epoch 51/150
1051/1051 [==============================] - 0s 160us/sample - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.1251 - val_mean_squared_error: 0.1251
Epoch 52/150
1051/1051 [==============================] - 0s 134us/sample - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 53/150
1051/1051 [==============================] - 0s 153us/sample - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.1125 - val_mean_squared_error: 0.1125
Epoch 54/150
1051/1051 [==============================] - 0s 153us/sample - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.1115 - val_mean_sq

1051/1051 [==============================] - 0s 164us/sample - loss: 7.0929e-04 - mean_squared_error: 7.0929e-04 - val_loss: 0.1025 - val_mean_squared_error: 0.1025
Epoch 98/150
1051/1051 [==============================] - 0s 163us/sample - loss: 5.8779e-04 - mean_squared_error: 5.8779e-04 - val_loss: 0.0988 - val_mean_squared_error: 0.0988
Epoch 99/150
1051/1051 [==============================] - 0s 261us/sample - loss: 4.0878e-04 - mean_squared_error: 4.0878e-04 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 100/150
1051/1051 [==============================] - 0s 221us/sample - loss: 3.4171e-04 - mean_squared_error: 3.4171e-04 - val_loss: 0.1001 - val_mean_squared_error: 0.1001
Epoch 101/150
1051/1051 [==============================] - 0s 143us/sample - loss: 2.5076e-04 - mean_squared_error: 2.5076e-04 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 102/150
1051/1051 [==============================] - 0s 149us/sample - loss: 2.1836e-04 - mean_squared_error: 2.183

Epoch 144/150
1051/1051 [==============================] - 0s 122us/sample - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 0.0984 - val_mean_squared_error: 0.0984
Epoch 145/150
1051/1051 [==============================] - 0s 133us/sample - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.1122 - val_mean_squared_error: 0.1122
Epoch 146/150
1051/1051 [==============================] - 0s 148us/sample - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0976 - val_mean_squared_error: 0.0976
Epoch 147/150
1051/1051 [==============================] - 0s 151us/sample - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.1123 - val_mean_squared_error: 0.1123
Epoch 148/150
1051/1051 [==============================] - 0s 148us/sample - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0987 - val_mean_squared_error: 0.0987
Epoch 149/150
1051/1051 [==============================] - 0s 147us/sample - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.1069 - val_m

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [42]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 71us/sample - loss: 0.0020 - mean_squared_error: 0.0020


[0.0019712512413275515, 0.001971251]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [43]:
# Evaluate the model on validate data
adam_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 108us/sample - loss: 0.1577 - mean_squared_error: 0.1577


[0.15773536979335628, 0.15773536]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [44]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 61us/sample - loss: 0.1577 - mean_squared_error: 0.1577


[0.15773536979335628, 0.15773536]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [45]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
rms = np.sqrt(np.mean((y_test - y_test_pred)**2))
rms

SalePrice    31210.162066
dtype: float64

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.